In [4]:
import pandas as pd
import numpy as np

# Find rows that do contain non-numerical spurious data.
raw_data = pd.read_excel("data/ouse93-96-raw-data.xlsx")
raw_data[~raw_data.applymap(np.isreal).all(1)]

,Unnamed: 0,Mean Daily Flow - Cumecs,Unnamed: 2,Unnamed: 3,Unnamed: 4,Daily Rainfall Total - mm,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11
0,NaT,Crakehill,Skip Bridge,Westwick,Skelton,Arkengarthdale,East Cowton,Malham Tarn,Snaizeholme,NaN,NaN,You are to predict the mean daily flow at Skel...
1,1993-01-01,10.4,4.393,9.291,26.1,0,0,0,4,NaN,NaN,(Cumecs = m^3/s)
97,1993-04-07,22.9,4.159,26.603,a,2.4,0,0,3.2,NaN,NaN,NaN
790,1995-03-01,80.6,19.096,100.761,174,17.6,#,51.2,21.6,NaN,NaN,NaN
1135,1996-02-09,12.7,a,7.576,20.58,0,37.6,12,14.4,NaN,NaN,NaN
